In [1]:
!pip install Word2vec
import pandas as pd
# Import Statements
from sklearn.pipeline import Pipeline
from sklearn.datasets import fetch_20newsgroups
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import SGDClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
import gensim
from gensim.models import Word2Vec, KeyedVectors
import pandas 
import nltk
from sklearn.preprocessing import StandardScaler
import spacy

In [2]:
data = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
data.shape, test.shape

((2586, 3), (288, 2))

In [3]:
# For the final submission
data = pd.merge(data, test,how='left')

In [4]:
data.head()

,id,description,category
0,1,A marriage of 13 and 18 year old bourbons. A m...,2
1,2,There have been some legendary Bowmores from t...,1
2,3,This bottling celebrates master distiller Park...,2
3,4,What impresses me most is how this whisky evol...,1
4,9,"A caramel-laden fruit bouquet, followed by une...",2


In [5]:
data.shape

(2586, 3)

## TfidfVectorizer with Random Forest classifier

In [220]:
# Create Pipeline Components

vect = TfidfVectorizer(stop_words='english')
rfc = RandomForestClassifier()

In [221]:
# Define the Pipeline
pipe = Pipeline([('vect', vect), ('clf', rfc)])

In [224]:

parameters = {
    'vect__max_df': (0.5, 0.75, 1.0),
    'vect__min_df': (.02, .05),
    'vect__max_features': (100, 500,1000),
    'clf__n_estimators':(5, 10, 50, 100),
    'clf__max_depth':(5,10,15,20),
    
}

grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(data['description'], data['category'])

Fitting 5 folds for each of 288 candidates, totalling 1440 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    4.6s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   15.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   38.6s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 1440 out of 1440 | elapsed:  2.5min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...obs=None,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'vect__max_df': (0.5, 0.75, 1.0), 'vect__min_df': (0.02, 0.05), 'vect__max_features': (100, 500, 1000), 'clf__n_estimators': (5, 10, 50, 100), 'clf__max_depth': (5, 10, 15, 20)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [225]:
grid_search.best_params_

{'clf__max_depth': 20,
 'clf__n_estimators': 100,
 'vect__max_df': 1.0,
 'vect__max_features': 1000,
 'vect__min_df': 0.02}

In [226]:
grid_search.best_score_

0.88553750966744

## Submission process

In [ ]:
ypred = grid_search.predict(test.description)

In [ ]:
#submission['category'] = y_pred
#submission['category'] = submission['category'].astype(int)
#submission.head()

## Best Score so far, TfidVectorizer with SGDC optimized parameters

In [6]:

# Create Pipeline

vect = TfidfVectorizer(stop_words='english')
scaler = StandardScaler(with_mean=False, with_std=False)
sgdc = SGDClassifier(alpha=0.0007, 
                     average=False, 
                     class_weight='balanced',
                     early_stopping=False, 
                     epsilon=0.01, 
                     eta0=0.1, 
                     fit_intercept=True,
                     l1_ratio=0.15, 
                     learning_rate='optimal', 
                     loss='hinge', 
                     max_iter=500,
                     n_iter_no_change=8, 
                     n_jobs=None,  
                     penalty='l2', 
                     power_t=0.5,
                     random_state=69, 
                     shuffle=True, 
                     tol=0.0015, 
                     validation_fraction=0.05,
                     verbose=0, 
                     warm_start=True)

pipe = Pipeline([('vect', vect), 
                 ('scaler', scaler),
                 ('clf', sgdc)])
# Fit Pipeline
pipe.fit(data.description, data.category)

Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=69, shuffle=True, tol=0.0015,
       validation_fraction=0.05, verbose=0, warm_start=True))])

In [7]:

parameters = {
    'clf__tol' : (.0015, 1, 2, 3, 4, .05, .005)

}

In [8]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)


In [9]:
grid_search.fit(data.description, data.category)


Fitting 5 folds for each of 7 candidates, totalling 35 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  35 out of  35 | elapsed:    5.3s finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm='l2', preprocessor=None, smooth_idf=True,
...m_state=69, shuffle=True, tol=0.0015,
       validation_fraction=0.05, verbose=0, warm_start=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__tol': (0.0015, 1, 2, 3, 4, 0.05, 0.005)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [10]:

grid_search.best_params_

{'clf__tol': 0.0015}

In [11]:

grid_search.best_score_

0.9404485692188709

In [13]:
pred = grid_search.predict(test['description'])

In [14]:

submission = pd.DataFrame({'id': test['id'], 'category':pred})
submission['category'] = submission['category'].astype('int64')

In [15]:
# Make Sure the Category is an Integer
submission.head()

,id,category
0,955,2
1,3532,2
2,1390,4
3,1024,1
4,1902,1


In [18]:
submission.to_csv('./data/submission2.csv', index=False)

## Trying Stuff

In [486]:

# Create Pipeline

vect = CountVectorizer(stop_words='english')

scaler = StandardScaler(with_mean=False, with_std=False)
sgdc = SGDClassifier(alpha=0.0007, 
                     average=False, 
                     class_weight='balanced',
                     early_stopping=False, 
                     epsilon=0.1, 
                     eta0=0.1, 
                     fit_intercept=True,
                     l1_ratio=0.15, 
                     learning_rate='optimal', 
                     loss='hinge', 
                     max_iter=500,
                     n_iter_no_change=8, 
                     n_jobs=None,  
                     penalty='l2', 
                     power_t=0.5,
                     random_state=69, 
                     shuffle=True, 
                     tol=0.0015, 
                     validation_fraction=0.05,
                     verbose=0, 
                     warm_start=True)

pipe = Pipeline([('vect', vect), 
                 ('scaler', scaler),
                 ('clf', sgdc)])
# Fit Pipeline
pipe.fit(data.description, data.category)

/Users/Inceptive/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...m_state=69, shuffle=True, tol=0.0015,
       validation_fraction=0.05, verbose=0, warm_start=True))])

In [487]:

parameters = {
    'clf__tol' : (.0015, 2)

}

In [488]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)


In [489]:
grid_search.fit(data.description, data.category)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:    2.9s remaining:    1.9s
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:    3.1s finished
/Users/Inceptive/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        ...m_state=69, shuffle=True, tol=0.0015,
       validation_fraction=0.05, verbose=0, warm_start=True))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__tol': (0.0015, 2)}, pre_dispatch='2*n_jobs',
       refit=True, return_train_score='warn', scoring=None, verbose=1)

In [490]:

grid_search.best_params_

{'clf__tol': 2}

In [491]:

grid_search.best_score_

0.922660479505027

## SVD + XGBClassifier

In [110]:
svd = TruncatedSVD(n_components=100,
                  algorithm = 'randomized',
                  n_iter=10)

In [111]:
from xgboost import XGBClassifier
xgb = XGBClassifier(n_jobs=11)
lsi = Pipeline([('vect', vect), ('svd', svd)])

pipe = Pipeline([('lsi', lsi), ('clf', xgb)])

In [112]:
pipe.fit(data.description, data.category)

Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))])

In [124]:
parameters = {
    'clf__n_estimators' : (1000, 2000),
  

}

In [125]:
grid_search = GridSearchCV(pipe,parameters, cv=5, n_jobs=-1, verbose=1)


In [126]:
grid_search.fit(data.description, data.category)


Fitting 5 folds for each of 2 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  1.9min remaining:  1.3min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed:  2.6min finished


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('lsi', Pipeline(memory=None,
     steps=[('vect', TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.float64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), norm=...
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=None,
       subsample=1, verbosity=1))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'clf__n_estimators': (1000, 2000)},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=1)

In [127]:

grid_search.best_params_

{'clf__n_estimators': 1000}

In [128]:

grid_search.best_score_

0.9245939675174014

## TFidVectorizer, TruncatedSVD for dimensionality reduction, SGDClassifier for classification

In [167]:
vect = TfidfVectorizer(stop_words='english')

svd = TruncatedSVD(n_components = 100,
                   algorithm='randomized',
                   n_iter=10, 
                   random_state=69)
sgdc = SGDClassifier(max_iter = 500,
                     n_iter_no_change = 2,
                     tol = 0.0015,
                     validation_fraction = 0.05,
                     random_state=69)

pipe = Pipeline([('vect', vect), 
                  ('svd', svd),
                  ('sgdc', sgdc)])

parameters = {
    'svd__n_components': (100, 300, 1000),
    'sgdc__max_iter': (300, 1000, 3000),
}
clf = GridSearchCV(pipe, parameters, cv=5, n_jobs=7)

In [19]:
#clf.fit(data['description'], data['category'])
''' Takes too long to run'''

' Takes too long to run'

In [ ]:
clf.best_params_

In [ ]:
clf.best_score_


## CountVectorizer, TruncatedSVD for dimensionality reduction, SGDClassifier for classification

In [218]:
vect = CountVectorizer(stop_words='english')

svd = TruncatedSVD(n_components = 1000,
                   algorithm='randomized',
                   n_iter=10, 
                   random_state=69,
                   tol=0)

sgdc = SGDClassifier(max_iter = 300,
                     n_iter_no_change = 2,
                     tol = 0.0015,
                     validation_fraction = 0.05,
                     random_state=69)

pipe = Pipeline([('vect', vect), 
                  ('svd', svd),
                  ('sgdc', sgdc)])


clf = GridSearchCV(pipe, parameters, cv=5, n_jobs=7)

In [20]:
#clf.fit(data['description'], data['category'])

In [217]:
clf.best_score_


0.9307811291569992